In [1]:
from ai import ai14s, ai_abs_dls
from marubatsu import Marubatsu

mb = Marubatsu()
eval_params = {"minimax": True}
result = ai_abs_dls(mb, eval_func=ai14s, eval_params=eval_params, 
                    use_tt=True, maxdepth=8, analyze=True)
print(len(result["tt"]))

3219


In [2]:
from marubatsu import ListBoard

def getmark(self, x, y):
    return self.board[x][y]

ListBoard.getmark = getmark

In [3]:
from marubatsu import Markpat
from collections import defaultdict

def is_winner(self, player, last_move):
    x, y = last_move
    if self.count_linemark:
        if self.rowcount[player][y] == self.BOARD_SIZE or \
        self.colcount[player][x] == self.BOARD_SIZE:
            return True
        # 左上から右下方向の判定
        if x == y and self.diacount[player][0] == self.BOARD_SIZE:
            return True
        # 右上から左下方向の判定
        if x + y == self.BOARD_SIZE - 1 and \
            self.diacount[player][1] == self.BOARD_SIZE:
            return True
    else:
        if self.is_same(player, x=0, y=y, dx=1, dy=0) or \
        self.is_same(player, x=x, y=0, dx=0, dy=1):
            return True
        # 左上から右下方向の判定
        if x == y and self.is_same(player, x=0, y=0, dx=1, dy=1):
            return True
        # 右上から左下方向の判定
        if x + y == self.BOARD_SIZE - 1 and \
            self.is_same(player, x=self.BOARD_SIZE - 1, y=0, dx=-1, dy=1):
            return True
    
    # どの一直線上にも配置されていない場合は、player は勝利していないので False を返す
    return False

ListBoard.is_winner = is_winner

def is_same(self, mark, x, y, dx, dy):
    text_list = [self.board[x + i * dx][y + i * dy]
                 for i in range(self.BOARD_SIZE)]
    line_text = "".join(text_list)
    return line_text == mark * self.BOARD_SIZE

ListBoard.is_same = is_same

def count_markpats(self, turn, last_turn):
    markpats = defaultdict(int)
    
    if self.count_linemark:
        for countdict in [self.rowcount, self.colcount, self.diacount]:
            for circlecount, crosscount in zip(countdict[Marubatsu.CIRCLE], countdict[Marubatsu.CROSS]):
                emptycount = self.BOARD_SIZE - circlecount - crosscount
                if last_turn == Marubatsu.CIRCLE:
                    markpats[(circlecount, crosscount, emptycount)] += 1
                else:
                    markpats[(crosscount, circlecount, emptycount)] += 1
    else:
        # 横方向と縦方向の判定
        for i in range(self.BOARD_SIZE):
            count = self.count_marks(turn, last_turn, x=0, y=i, dx=1, dy=0, datatype="tuple")
            markpats[count] += 1
            count = self.count_marks(turn, last_turn, x=i, y=0, dx=0, dy=1, datatype="tuple")
            markpats[count] += 1
        # 左上から右下方向の判定
        count = self.count_marks(turn, last_turn, x=0, y=0, dx=1, dy=1, datatype="tuple")
        markpats[count] += 1
        # 右上から左下方向の判定
        count = self.count_marks(turn, last_turn, x=self.BOARD_SIZE - 1, y=0, dx=-1, dy=1, datatype="tuple")
        markpats[count] += 1

    return markpats  

ListBoard.count_markpats = count_markpats

def count_marks(self, turn, last_turn, x, y, dx, dy, datatype="dict"):    
    count = defaultdict(int)
    for _ in range(self.BOARD_SIZE):
        count[self.board[x][y]] += 1
        x += dx
        y += dy

    if datatype == "dict":
        return count
    else:
        return Markpat(count[last_turn], count[turn], count[Marubatsu.EMPTY])  
    
ListBoard.count_marks = count_marks

In [4]:
from util import benchmark

boardclass = ListBoard
for count_linemark in [False, True]:
    print(f"boardclass: {boardclass.__name__}, count_linemark {count_linemark}")
    benchmark(mbparams={"boardclass": boardclass, "count_linemark": count_linemark})
    print()

boardclass: ListBoard, count_linemark False
ai2 VS ai2


100%|██████████| 50000/50000 [00:03<00:00, 14579.71it/s]


count     win    lose    draw
o       29454   14352    6194
x       14208   29592    6200
total   43662   43944   12394

ratio     win    lose    draw
o       58.9%   28.7%   12.4%
x       28.4%   59.2%   12.4%
total   43.7%   43.9%   12.4%

ai14s VS ai2


100%|██████████| 50000/50000 [00:44<00:00, 1113.82it/s]


count     win    lose    draw
o       49446       0     554
x       44043       0    5957
total   93489       0    6511

ratio     win    lose    draw
o       98.9%    0.0%    1.1%
x       88.1%    0.0%   11.9%
total   93.5%    0.0%    6.5%

ai_abs_dls
 18.0 ms ±   1.6 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)

boardclass: ListBoard, count_linemark True
ai2 VS ai2


100%|██████████| 50000/50000 [00:03<00:00, 15309.97it/s]


count     win    lose    draw
o       29454   14352    6194
x       14208   29592    6200
total   43662   43944   12394

ratio     win    lose    draw
o       58.9%   28.7%   12.4%
x       28.4%   59.2%   12.4%
total   43.7%   43.9%   12.4%

ai14s VS ai2


100%|██████████| 50000/50000 [00:25<00:00, 1984.76it/s]


count     win    lose    draw
o       49446       0     554
x       44043       0    5957
total   93489       0    6511

ratio     win    lose    draw
o       98.9%    0.0%    1.1%
x       88.1%    0.0%   11.9%
total   93.5%    0.0%    6.5%

ai_abs_dls
 17.4 ms ±   0.7 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)



In [5]:
from marubatsu import List1dBoard

def is_same(self, mark, x, y, dx, dy):
    text_list = [self.board[y + i * dy + (x + i * dx) * self.BOARD_SIZE]
                 for i in range(self.BOARD_SIZE)]
    line_text = "".join(text_list)
    return line_text == mark * self.BOARD_SIZE

List1dBoard.is_same = is_same

def count_marks(self, turn, last_turn, x, y, dx, dy, datatype="dict"):    
    count = defaultdict(int)
    for _ in range(self.BOARD_SIZE):
        count[self.board[y + x * self.BOARD_SIZE]] += 1
        x += dx
        y += dy

    if datatype == "dict":
        return count
    else:
        return Markpat(count[last_turn], count[turn], count[Marubatsu.EMPTY])  
    
List1dBoard.count_marks = count_marks

In [6]:
def getmark(self, x, y):
    return self.board[y + x * self.BOARD_SIZE]

List1dBoard.getmark = getmark

In [7]:
boardclass = List1dBoard
for count_linemark in [False, True]:
    print(f"boardclass: {boardclass.__name__}, count_linemark {count_linemark}")
    benchmark(mbparams={"boardclass": boardclass, "count_linemark": count_linemark})
    print()

boardclass: List1dBoard, count_linemark False
ai2 VS ai2


100%|██████████| 50000/50000 [00:04<00:00, 11818.58it/s]


count     win    lose    draw
o       29454   14352    6194
x       14208   29592    6200
total   43662   43944   12394

ratio     win    lose    draw
o       58.9%   28.7%   12.4%
x       28.4%   59.2%   12.4%
total   43.7%   43.9%   12.4%

ai14s VS ai2


100%|██████████| 50000/50000 [00:47<00:00, 1050.48it/s]


count     win    lose    draw
o       49446       0     554
x       44043       0    5957
total   93489       0    6511

ratio     win    lose    draw
o       98.9%    0.0%    1.1%
x       88.1%    0.0%   11.9%
total   93.5%    0.0%    6.5%

ai_abs_dls
 20.9 ms ±   1.1 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)

boardclass: List1dBoard, count_linemark True
ai2 VS ai2


100%|██████████| 50000/50000 [00:04<00:00, 10430.42it/s]


count     win    lose    draw
o       29454   14352    6194
x       14208   29592    6200
total   43662   43944   12394

ratio     win    lose    draw
o       58.9%   28.7%   12.4%
x       28.4%   59.2%   12.4%
total   43.7%   43.9%   12.4%

ai14s VS ai2


100%|██████████| 50000/50000 [00:31<00:00, 1591.74it/s]


count     win    lose    draw
o       49446       0     554
x       44043       0    5957
total   93489       0    6511

ratio     win    lose    draw
o       98.9%    0.0%    1.1%
x       88.1%    0.0%   11.9%
total   93.5%    0.0%    6.5%

ai_abs_dls
 24.3 ms ±   2.9 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)



In [8]:
class Move:
    def __init__(self, move, board_size):
        self.move = move
        self.board_size = board_size

    def __getitem__(self, key):
        if key == 0:
            return self.move // self.board_size
        elif key == 1:
            return self.move % self.board_size
        else:
            raise IndexError
        
move = Move(5, 3)
x, y = move
print(x, y)
%timeit x, y = move

1 2
615 ns ± 6.13 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [9]:
def move_to_xy(move):
    return move // 3, move % 3

move = 5
x, y = move_to_xy(move)
print(x, y)
%timeit x, y = move_to_xy(move)

1 2
106 ns ± 3.07 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [10]:
def calc_legal_moves(self):
    legal_moves = [move for move in range(self.BOARD_SIZE ** 2) 
                        if self.board[move] == Marubatsu.EMPTY]
    return legal_moves
 
List1dBoard.calc_legal_moves = calc_legal_moves 

In [11]:
def getmark_by_move(self, move):
    return self.board[move]    

List1dBoard.getmark_by_move = getmark_by_move

def setmark_by_move(self, move, mark):
    if self.count_linemark:
        x, y = move // self.BOARD_SIZE, move % self.BOARD_SIZE
        if mark != Marubatsu.EMPTY:
            diff = 1
            changedmark = mark
        else:
            diff = -1
            changedmark = self.board[move]
        self.colcount[changedmark][x] += diff
        self.rowcount[changedmark][y] += diff
        if x == y:
            self.diacount[changedmark][0] += diff
        if x + y == self.BOARD_SIZE - 1:
            self.diacount[changedmark][1] += diff
    self.board[move] = mark
    
List1dBoard.setmark_by_move = setmark_by_move

In [12]:
def xy_to_move(self, x, y):
    return y + x * self.BOARD_SIZE
List1dBoard.xy_to_move = xy_to_move

def move_to_xy(self, move):
    return move // self.BOARD_SIZE, move % self.BOARD_SIZE

List1dBoard.move_to_xy = move_to_xy

def move_to_xy(self, move):
    return move

ListBoard.move_to_xy = move_to_xy


In [13]:
def is_winner(self, player, last_move):
    x, y = last_move // self.BOARD_SIZE, last_move % self.BOARD_SIZE
    if self.count_linemark:
        if self.rowcount[player][y] == self.BOARD_SIZE or \
        self.colcount[player][x] == self.BOARD_SIZE:
            return True
        # 左上から右下方向の判定
        if x == y and self.diacount[player][0] == self.BOARD_SIZE:
            return True
        # 右上から左下方向の判定
        if x + y == self.BOARD_SIZE - 1 and \
            self.diacount[player][1] == self.BOARD_SIZE:
            return True
    else:
        if self.is_same(player, x=0, y=y, dx=1, dy=0) or \
        self.is_same(player, x=x, y=0, dx=0, dy=1):
            return True
        # 左上から右下方向の判定
        if x == y and self.is_same(player, x=0, y=0, dx=1, dy=1):
            return True
        # 右上から左下方向の判定
        if x + y == self.BOARD_SIZE - 1 and \
            self.is_same(player, x=self.BOARD_SIZE - 1, y=0, dx=-1, dy=1):
            return True
    
    # どの一直線上にも配置されていない場合は、player は勝利していないので False を返す
    return False

List1dBoard.is_winner = is_winner

In [14]:
def calc_same_boardtexts(mb, move=None):
    data = [ [ 0,  0,  1, 1, -1,  0,  1,  0, -1, 0,  1,  0],
             [ 1, -1,  0, 1,  0, -1] * 2,
             [ 1,  0, -1, 0,  1,  0,  0,  0,  1, 1, -1,  0],
             [ 1, -1,  0, 0,  0,  1] * 2,
             [ 0,  1,  0, 1,  0, -1] * 2,
             [ 1,  0, -1, 1, -1,  0] * 2,
             [ 0,  0,  1, 0,  1,  0] * 2, ]
    if move is None:
        boardtexts = set([mb.board_to_str()])
    else:
        boardtexts = { mb.board_to_str(): move }
    for xa, xb, xc, ya, yb, yc, xa2, xb2, xc2, ya2, yb2, yc2 in data:
        txt = ""
        for x in range(mb.BOARD_SIZE):
            for y in range(mb.BOARD_SIZE):
                txt += mb.board.getmark(xa * 2 + xb * x + xc * y, ya * 2 + yb * x + yc * y)
        if move is None:
            boardtexts.add(txt)
        else:
            x, y = mb.board.move_to_xy(move)
            x, y = xa2 * 2 + xb2 * x + xc2 * y, ya2 * 2 + yb2 * x + yc2 * y
            boardtexts[txt] = mb.board.xy_to_move(x, y)
    return boardtexts

In [15]:
from ai import ai_by_mmscore, dprint
from time import perf_counter

@ai_by_mmscore
def ai_abs_dls(mb, debug=False, timelimit_pc=None, maxdepth=1,
               eval_func=None, eval_params={}, use_tt=False,
               tt=None, tt_for_mo=None):
    count = 0
    def ab_search(mborig, depth, tt, alpha=float("-inf"), beta=float("inf")):
        nonlocal count
        if timelimit_pc is not None and perf_counter() >= timelimit_pc:
            raise RuntimeError("time out")
        
        count += 1
        if mborig.status != Marubatsu.PLAYING or depth == maxdepth:
            return eval_func(mborig, calc_score=True, **eval_params)
        
        if use_tt:
            boardtxt = mborig.board_to_str()
            if boardtxt in tt:
                lower_bound, upper_bound, _ = tt[boardtxt]
                if lower_bound == upper_bound:
                    return lower_bound
                elif upper_bound <= alpha:
                    return upper_bound
                elif beta <= lower_bound:
                    return lower_bound
                else:
                    alpha = max(alpha, lower_bound)
                    beta = min(beta, upper_bound)
            else:
                lower_bound = min_score
                upper_bound = max_score
        
        alphaorig = alpha
        betaorig = beta

        legal_moves = mborig.calc_legal_moves()
        if tt_for_mo is not None:
            if not use_tt:            
                boardtxt = mborig.board_to_str()
            if boardtxt in tt_for_mo:
                _, _, bestmove = tt_for_mo[boardtxt]
                index = legal_moves.index(bestmove)
                legal_moves[0], legal_moves[index] = legal_moves[index], legal_moves[0]        
        if mborig.turn == Marubatsu.CIRCLE:
            score = float("-inf")
            for move in legal_moves:
                mborig.move(move)
                abscore = ab_search(mborig, depth + 1, tt, alpha, beta)
                mborig.unmove()
                if abscore > score:
                    bestmove = move
                score = max(score, abscore)
                if score >= beta:
                    break
                alpha = max(alpha, score)
        else:
            score = float("inf")
            for move in legal_moves:
                mborig.move(move)
                abscore = ab_search(mborig, depth + 1, tt, alpha, beta)
                mborig.unmove()
                if abscore < score:
                    bestmove = move
                score = min(score, abscore)
                if score <= alpha:
                    break
                beta = min(beta, score)   
            
#        from util import calc_same_boardtexts

        if use_tt:
            boardtxtlist = calc_same_boardtexts(mborig, bestmove)
            if score <= alphaorig:
                upper_bound = score
            elif score < betaorig:
                lower_bound = score
                upper_bound = score
            else:
                lower_bound = score
            for boardtxt, move in boardtxtlist.items():
                tt[boardtxt] = (lower_bound, upper_bound, move)
        return score
                
    min_score = float("-inf")
    max_score = float("inf")
    
    if tt is None:
        tt = {}
    score = ab_search(mb, depth=0, tt=tt, alpha=min_score, beta=max_score)
    dprint(debug, "count =", count)
    return score, count

In [16]:
from ai import ai_match, ai2, ai14s

match_num = 50000
for boardclass in [ListBoard, List1dBoard]:
    for count_linemark in [False, True]:
        print(f"boardclass: {boardclass.__name__}, count_linemark {count_linemark}")
        mbparams={"boardclass": boardclass, "count_linemark": count_linemark}
        ai_match(ai=[ai2, ai2], match_num=match_num, mbparams=mbparams)   
        ai_match(ai=[ai14s, ai2], match_num=match_num, mbparams=mbparams)

        mb = Marubatsu(**mbparams)
        eval_params = {"minimax": True}
        print("ai_abs_dls")
        %timeit ai_abs_dls(mb, eval_func=ai14s, eval_params=eval_params, use_tt=True, maxdepth=8)
        print()

boardclass: ListBoard, count_linemark False
ai2 VS ai2


100%|██████████| 50000/50000 [00:03<00:00, 13986.04it/s]


count     win    lose    draw
o       29297   14378    6325
x       14635   29094    6271
total   43932   43472   12596

ratio     win    lose    draw
o       58.6%   28.8%   12.7%
x       29.3%   58.2%   12.5%
total   43.9%   43.5%   12.6%

ai14s VS ai2


100%|██████████| 50000/50000 [00:45<00:00, 1108.16it/s]


count     win    lose    draw
o       49491       0     509
x       44164       0    5836
total   93655       0    6345

ratio     win    lose    draw
o       99.0%    0.0%    1.0%
x       88.3%    0.0%   11.7%
total   93.7%    0.0%    6.3%

ai_abs_dls
17.3 ms ± 867 μs per loop (mean ± std. dev. of 7 runs, 10 loops each)

boardclass: ListBoard, count_linemark True
ai2 VS ai2


...

count     win    lose    draw
o       49486       0     514
x       44141       0    5859
total   93627       0    6373

ratio     win    lose    draw
o       99.0%    0.0%    1.0%
x       88.3%    0.0%   11.7%
total   93.6%    0.0%    6.4%

ai_abs_dls
17.2 ms ± 258 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)

boardclass: List1dBoard, count_linemark True
ai2 VS ai2


100%|██████████| 50000/50000 [00:02<00:00, 17326.73it/s]


count     win    lose    draw
o       29330   14288    6382
x       14224   29385    6391
total   43554   43673   12773

ratio     win    lose    draw
o       58.7%   28.6%   12.8%
x       28.4%   58.8%   12.8%
total   43.6%   43.7%   12.8%

ai14s VS ai2


100%|██████████| 50000/50000 [00:24<00:00, 2029.60it/s]


count     win    lose    draw
o       49470       0     530
x       44279       0    5721
total   93749       0    6251

ratio     win    lose    draw
o       98.9%    0.0%    1.1%
x       88.6%    0.0%   11.4%
total   93.7%    0.0%    6.3%

ai_abs_dls
19.2 ms ± 1.77 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)



In [ ]:
from ai import ai_by_score

@ai_by_score
def ai14s(mb, debug=False, score_victory=300, score_sure_victory=200, \
          score_defeat=-100, score_special=100, score_201=2, \
          score_102=0.5, score_012=-1):
    # 評価値の合計を計算する変数を 0 で初期化する
    score = 0        

    # 自分が勝利している場合
    if mb.status == mb.last_turn:
        return score_victory

    markpats = mb.count_markpats()
    if debug:
        pprint(markpats)
    # 相手が勝利できる場合は評価値を加算する
    if markpats[Markpat(last_turn=0, turn=2, empty=1)] > 0:
        score = score_defeat * markpats[Markpat(last_turn=0, turn=2, empty=1)]
    # 次の自分の手番で自分が必ず勝利できる場合
    elif markpats[Markpat(last_turn=2, turn=0, empty=1)] >= 2:
        return score_sure_victory
    
    # 斜め方向に 〇×〇 が並び、いずれかの辺の 1 つのマスのみに × が配置されている場合
    if mb.move_count == 4 and \
        mb.board.getmark(1, 1) == Marubatsu.CROSS and \
        (mb.board.getmark(0, 0) == mb.board.getmark(2, 2) == Marubatsu.CIRCLE or \
        mb.board.getmark(2, 0) == mb.board.getmark(0, 2) == Marubatsu.CIRCLE) and \
        (mb.board.getmark(1, 0) == Marubatsu.CROSS or \
        mb.board.getmark(0, 1) == Marubatsu.CROSS or \
        mb.board.getmark(2, 1) == Marubatsu.CROSS or \
        mb.board.getmark(1, 2) == Marubatsu.CROSS):
        return score_special    

    # 次の自分の手番で自分が勝利できる場合は評価値に score_201 を加算する
    if markpats[Markpat(last_turn=2, turn=0, empty=1)] == 1:
        score += score_201
    # 「自 1 敵 0 空 2」1 つあたり score_102 だけ、評価値を加算する
    score += markpats[Markpat(last_turn=1, turn=0, empty=2)] * score_102
    # 「自 0 敵 1 空 2」1 つあたり score_201 だけ、評価値を減算する
    score += markpats[Markpat(last_turn=0, turn=1, empty=2)] * score_012
    
    # 計算した評価値を返す
    return score

In [18]:
for boardclass in [ListBoard, List1dBoard]:
    for count_linemark in [False, True]:
        print(f"boardclass: {boardclass.__name__}, count_linemark {count_linemark}")
        mbparams={"boardclass": boardclass, "count_linemark": count_linemark}
        ai_match(ai=[ai2, ai2], match_num=match_num, mbparams=mbparams)   
        ai_match(ai=[ai14s, ai2], match_num=match_num, mbparams=mbparams)

        mb = Marubatsu(**mbparams)
        eval_params = {"minimax": True}
        print("ai_abs_dls")
        %timeit ai_abs_dls(mb, eval_func=ai14s, eval_params=eval_params, use_tt=True, maxdepth=8)
        print()

boardclass: ListBoard, count_linemark False
ai2 VS ai2


100%|██████████| 50000/50000 [00:03<00:00, 14916.51it/s]


count     win    lose    draw
o       29252   14312    6436
x       14337   29339    6324
total   43589   43651   12760

ratio     win    lose    draw
o       58.5%   28.6%   12.9%
x       28.7%   58.7%   12.6%
total   43.6%   43.7%   12.8%

ai14s VS ai2


100%|██████████| 50000/50000 [00:44<00:00, 1116.53it/s]


count     win    lose    draw
o       49461       0     539
x       44217       0    5783
total   93678       0    6322

ratio     win    lose    draw
o       98.9%    0.0%    1.1%
x       88.4%    0.0%   11.6%
total   93.7%    0.0%    6.3%

ai_abs_dls
17.4 ms ± 298 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)

boardclass: ListBoard, count_linemark True
ai2 VS ai2


...

count     win    lose    draw
o       49473       0     527
x       44077       0    5923
total   93550       0    6450

ratio     win    lose    draw
o       98.9%    0.0%    1.1%
x       88.2%    0.0%   11.8%
total   93.5%    0.0%    6.5%

ai_abs_dls
16.7 ms ± 277 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)

boardclass: List1dBoard, count_linemark True
ai2 VS ai2


100%|██████████| 50000/50000 [00:02<00:00, 17176.56it/s]


count     win    lose    draw
o       29201   14419    6380
x       14251   29435    6314
total   43452   43854   12694

ratio     win    lose    draw
o       58.4%   28.8%   12.8%
x       28.5%   58.9%   12.6%
total   43.5%   43.9%   12.7%

ai14s VS ai2


100%|██████████| 50000/50000 [00:23<00:00, 2152.38it/s]


count     win    lose    draw
o       49476       0     524
x       44249       0    5751
total   93725       0    6275

ratio     win    lose    draw
o       99.0%    0.0%    1.0%
x       88.5%    0.0%   11.5%
total   93.7%    0.0%    6.3%

ai_abs_dls
17.3 ms ± 349 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)



In [19]:
from ai import ai6s

mb = Marubatsu()
ai6s(mb)

AttributeError: 'Marubatsu' object has no attribute 'count_marks'

In [20]:
@ai_by_score
def ai6s(mb, debug):
    # 自分が勝利している場合は、評価値として 1 を返す
    if mb.status == mb.last_turn:
        return 1

    markpats = mb.count_markpats()
    # 相手が勝利できる場合は評価値として -1 を返す
    if markpats[Markpat(last_turn=0, turn=2, empty=1)] > 0:
        return -1
    # それ以外の場合は評価値として 0 を返す
    else:
        return 0

In [21]:
ai6s(mb)

(1, 0)